# 크롤링을 하려면 selenium, chromedriver, urllib이 필수적으로 필요하다.

chromedriver의 경우 Only local connections are allowed. 라는 이슈가 있기에, 서버에서는 사용 불가능하다. <br>
그리고 일반 크롬과 버전이 맞아야 사용 가능하므로, 사용전에 버전 확인을 하는 것이 좋다. <br>
따로 버전 확인을 하지 않더라도 에러가 다 알려주니 크게 신경쓰지 않아도 됨. <br>

[chromedriver.exe 다운받는 경로](https://chromedriver.chromium.org/downloads) <br>

(현재 주피터를 실행하는 경로에) 현재 내 pc의 chrome과 버전이 동일한 chromedriver.exe가 깔려만 있으면 잘 실행됨.

In [4]:
# pip install --upgrade [selenium]

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '[selenium]'
You should consider upgrading via the 'C:\Users\kelly\anaconda3\python.exe -m pip install --upgrade pip' command.


In [11]:
import sys, os
import requests #(네이버) api 이용할 때 쓰는 패키지
import selenium #크롤링 패키지

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
#

from pandas import DataFrame
from bs4 import BeautifulSoup #크롤링 패키지
import re #정규식 표현 모듈
from datetime import datetime
import pickle, progressbar, json, glob, time #pickle: 객체를 파일로 저장해주는 모듈, progressbar: 진행표시 바, json: 네이버 api, glob: 파일들 리스트 뽑을 때 사용
# https://wikidocs.net/83
from tqdm import tqdm #진행표시바
#

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

import numpy as np

import urllib.request #파싱을 위한 모듈

# 크롬창 열기

In [7]:
print('브라우저를 실행시킵니다(자동 제어)\n')
browser = webdriver.Chrome('chromedriver') #크롬창열기 # (현재 주피터를 실행하는 경로에) 현재 내 크롬 버전과 동일한 chromedriver.exe가 있으면 실행됨.
# https://beomi.github.io/2017/09/28/HowToMakeWebCrawler-Headless-Chrome/

news_url = 'https://www.kdca.go.kr/npt/biz/npp/ist/bass/bassSexdstnAgeStatsMain.do'

browser.get(news_url) #url 오픈
time.sleep(2)

# version issue
# SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 87
# Current browser version is 103.0.5060.114 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe

# solution
# https://codechacha.com/ko/selenium-chromedriver-version-error/

브라우저를 실행시킵니다(자동 제어)



# 옵션버튼 클릭

In [15]:
search_opt_box = browser.find_element_by_xpath('//*[@id="ui-id-3"]')
# 검색 옵션 버튼
search_opt_box.click() #버튼 클릭
time.sleep(0.02)

# 기간 선택
search_box = browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmStartDt"]')
search_box.send_keys('2001-01-01')
search_box.send_keys(Keys.RETURN)
time.sleep(0.02)

search_box = browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmEndDt"]')
search_box.send_keys('2001-01-31')
search_box.send_keys(Keys.RETURN)
time.sleep(0.02)

## 공통
# 감염병 명 선택
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[1]/div/button/div').click()
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[1]/div/div/ul/li[1]/label/input').click()
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/button/div').click()
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[1]/label/input').click()
time.sleep(0.02)

# 연령 선택
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm_ageType2"]')


# 지역 설정, 하나하나 클릭해야 하므로 Xpath의 패턴을 찾아서 for loop을 돌릴 수 있도록 한다.
## 시도
for 시도 in range(18):
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]').click()
    browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]/option[{시도+2}]').click()

    ## 군구
    if 시도[시도] == '경기':
        for 군구 in range(52): # 52
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            
            ## 통계작성버튼 클릭
            browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/input[2]').click()
            
            
            
    elif 시도[시도] == '서울' or 시도[시도] == '경북' or 시도[시도] == '경남':
        for 군구 in range(25): # 25
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()
            
    elif 시도[시도] == '전남':
        for 군구 in range(22): # 22
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()

    elif 시도[시도] == '강원' or 시도[시도] == '충남':
        for 군구 in range(18): # 18
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()

    elif 시도[시도] == '충북':
        for 군구 in range(17): # 17
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()
            
    elif 시도[시도] == '부산' or 시도[시도] == '전북':
        for 군구 in range(16): # 16
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()
    
    elif 시도[시도] == '인천':
        for 군구 in range(10): # 10
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()
    
    elif 시도[시도] == '대구':
        for 군구 in range(8): # 8
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()
    
    elif 시도[시도] == '광주' or 시도[시도] == '대전' or 시도[시도] == '울산':
        for 군구 in range(5): # 5
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()
            
    elif 시도[시도] == '제주':
        for 군구 in range(4): # 4
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()
            
    elif 시도[시도] == '세종':
        for 군구 in range(1): # 1
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            browser.find_element_by_xpath('').click()

In [ ]:
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]

In [ ]:
시도 = ['서울', '부산', '대구', '인천',
      '광주', '대전', '울산', '경기', '강원', 
      '충북', '충남', '전북', '전남', '경북',
      '경남', '제주', '세종']
len(시도) # 17

In [30]:
53-2+1

52

<각 지역 군구>
~~서울 : 1~25 ==> 25~~
~~부산 : 26~41 ==> 16~~
~~대구 : 42~49 ==> 8~~
~~인천 : 50~59 ==> 10~~

광주 : 60~64 ==> 5
대전 : 65~69 ==> 5
울산 : 70~74 ==> 5

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[47]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[48]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[53]
~~경기 : 75~120, 120(이게 2개임) 122, 123, 124, 125, 130 ==> 52 (2로 시작, 53으로 끝)~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[19]
~~강원 : 360~377 ==> 18 (2로 시작, 19로 끝)~~

~~충북 : 400~416 ==> 17~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[19]
~~충남 : 420~435 + 704, 705 ==> 18~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[17]
~~전북 : 550~565 ==> 16~~

~~전남 : 570~591 ==> 22~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[26]
~~경북 : 600~624 ==> 25~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[26]
~~경남 : 650~669, 706~710 ==>25~~

~~제주 : 700~703 ==> 4~~
~~세종 : 711 ==> 1~~

In [32]:
for 군구 in range(1): # 1
    print(군구)

0
